In [1]:
import os, time
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import pickle
import imageio
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.autograd import Variable
from PIL import Image
from numpy import array
import numpy as np
import copy
%matplotlib inline

In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

# 读取数据
wafer_data = pd.read_pickle("LSWMD.pkl")

In [3]:
# 提取不同类型数据
wafer_data = wafer_data.drop(['waferIndex'], axis = 1)
noType = wafer_data[wafer_data.failureType.str.len() == 0]
noneType = wafer_data[wafer_data.failureType == 'none']
wafer_data = wafer_data[wafer_data.failureType.str.len() > 0]
hasType = wafer_data[wafer_data.failureType != 'none']
hasType = hasType.reset_index(drop = True)


C:\Users\HP\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = libops.scalar_compare(x.ravel(), y, op)


In [4]:
noneType.waferMap[0].shape

(45, 48)

In [ ]:
# Tensor转图片
loader = transforms.Compose([
    transforms.ToTensor()])  

unloader = transforms.ToPILImage()

def tensor_to_PIL(tensor):
    image = tensor.cpu().clone()
    image = image.squeeze(0)
    image = unloader(image)
    return image

# 图片转tensor
transform1 = transforms.Compose([
    transforms.ToTensor(), 
    ]
)

In [5]:
# 重新调整图片大小
scaledimgs = []

for y in range(len(hasType)):
    img = Image.fromarray(hasType.waferMap[y])
    scimg = img.resize((32, 32), Image.ANTIALIAS)
    npscimg = np.array(scimg)
    scaledimgs.append(npscimg)

scaledimgs = np.array(scaledimgs)

In [ ]:
scaledimgs.shape

In [6]:
# 分开训练集和测试集的标签
scaledtrain = []
scaledtest = []
y_train = []
y_test = []
scall = []
y_all = []
traincount = 0
testcount = 0
allcount = 0

for y in range(len(hasType)):
#     scall.append(scaledimgs[y])
#     allcount += 1
#     y_all.append(hasType.failureType[y][:])
    if(hasType.trianTestLabel[y] == 'Training'):
        #if(type(scaledimgs[y]) is not int):
        scaledtrain.append(scaledimgs[y])
        traincount += 1
        y_train.append(hasType.failureType[y][:])
    elif(hasType.trianTestLabel[y] == 'Test'):
        scaledtest.append(scaledimgs[y])
        testcount += 1
        y_test.append(hasType.failureType[y][:])

nptrain = np.array(scaledtrain)
# npy_train = np.array(y_train)

nptest = np.array(scaledtest)
# npy_test = np.array(y_test)

# npall = np.array(scall)
# # npy = np.array(y_all)

print("traincount: ", traincount)
print("testcount: ", testcount)
print("allcount: ", allcount)

traincount:  17625
testcount:  7894
allcount:  0


In [36]:
fy_train

array([['Loc'],
       ['Edge-Loc'],
       ['Edge-Loc'],
       ...,
       ['Scratch'],
       ['Edge-Loc'],
       ['Loc']], dtype='<U9')

In [7]:
# 标签one hot编码
from sklearn import preprocessing

# fy_all = np.vstack(y_all)
fy_train = np.vstack(y_train)
fy_test = np.vstack(y_test)

one_hot = preprocessing.OneHotEncoder(sparse = False)

train_onehot = one_hot.fit_transform(fy_train)
test_onehot = one_hot.fit_transform(fy_test)
# all_onehot = one_hot.fit_transform(fy_all)
num_classes = len(train_onehot[0])

train_onehot = np.array(train_onehot)
test_onehot = np.array(test_onehot)
# all_onehot = np.array(all_onehot)

In [8]:
for x in nptrain:
    for i in range(32):
        for j in range(32):
            if x[i][j] < 1.5:
                x[i][j] = 0
            else:
                x[i][j] = 1

In [9]:
for x in nptest:
    for i in range(32):
        for j in range(32):
            if x[i][j] < 1.5:
                x[i][j] = 0
            else:
                x[i][j] = 1

In [ ]:
for x in npall:
    for i in range(32):
        for j in range(32):
            if x[i][j] < 1.5:
                x[i][j] = 0
            else:
                x[i][j] = 1

In [ ]:
a = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
b = np.array([[1],[2],[3],[4]])
np.random.seed(199)
np.random.shuffle(a)
np.random.seed(199)
np.random.shuffle(b)
print(a)
print(b)

In [ ]:
np.random.seed(200)
np.random.shuffle(nptrain)
np.random.seed(200)
np.random.shuffle(train_onehot)

In [ ]:
nptrain[0][10]

In [10]:
# 整理训练数据
x_train_tensor = torch.from_numpy(nptrain)
y_train_tensor = torch.from_numpy(train_onehot)
x_train_tensor = x_train_tensor.reshape([17625,1,32,32])


x_test_tensor = torch.from_numpy(nptest[:200])
y_test_tensor = torch.from_numpy(test_onehot[:200])
x_test_tensor = x_test_tensor.reshape([200,1,32,32])

In [ ]:
x_all_tensor = torch.from_numpy(npall/2)
y_all_tensor = torch.from_numpy(all_onehot)
x_all_tensor = x_all_tensor.reshape([25519,1,32,32])

In [ ]:
x_train_tensor[0][0][10]

In [11]:
torch_dataset = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)

In [12]:
torch_dataset_test = torch.utils.data.TensorDataset(x_test_tensor, y_test_tensor)

In [ ]:
torch_dataset_test[199]

In [ ]:
torch_dataset_all = torch.utils.data.TensorDataset(x_all_tensor, y_all_tensor)

In [ ]:
torch_dataset_all[0][0][0][10]

In [ ]:
conv1 = nn.Conv2d(1,16*2, 3, 2, 1)
conv1

In [27]:
class classifier(nn.Module):
    # initializers
    def __init__(self, d=128):
        super(classifier, self).__init__()
        self.conv1 = nn.Conv2d(1, d*2, 4, 2, 1)
        self.conv2 = nn.Conv2d(d*2, d*4, 4, 2, 1)
        self.conv2_bn = nn.BatchNorm2d(d*4)
        self.Linear1 = nn.Linear(d*256, 1024)
        self.Linear2 = nn.Linear(1024, 8)

    # weight_init
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)

    # forward method
    def forward(self, input):
#         x = F.leaky_relu(self.conv1(input), 0.2)
#         x = F.leaky_relu(self.conv2_bn(self.conv2(x)), 0.2)
#         x = x.view(-1,64*256)
#         x = F.leaky_relu(self.Linear1(x), 0.2)
#         x = F.softmax(self.Linear2(x), dim=1)
        x = F.relu(self.conv1(input))
        x = F.relu(self.conv2_bn(self.conv2(x)))
        x = x.view(-1,32*256)
        x = F.relu(self.Linear1(x))
        x = F.softmax(self.Linear2(x), dim=1)
  
        return x

In [ ]:
a = torch.tensor([[1,2,3.],[2,3,1.]])
s = F.softmax(a, dim=1)
s

In [19]:
def normal_init(m, mean, std):
    if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        m.weight.data.normal_(mean, std)
        m.bias.data.zero_()


In [28]:
# training parameters
batch_size = 32
lr = 0.0002
train_epoch = 5

train_loader = torch.utils.data.DataLoader(
    #datasets.MNIST('data', train=True, download=True, transform=transform),
    torch_dataset,
    batch_size=batch_size, shuffle=True)

In [21]:
test_loader = torch.utils.data.DataLoader(
    #datasets.MNIST('data', train=True, download=True, transform=transform),
    torch_dataset_test,
    batch_size=batch_size, shuffle=True)
testloader = test_loader

In [ ]:
for i, data in enumerate(train_loader, 0):
    inputs, labels = data
    print(inputs)
    print(labels)
    break

In [29]:
C = classifier(32)
C.weight_init(mean=0.0, std=0.02)

In [30]:
# Binary Cross Entropy loss
BCE_loss = nn.BCELoss()
CE_loss = nn.CrossEntropyLoss()

# Adam optimizer
C_optimizer = optim.Adam(C.parameters(), lr=lr, betas=(0.5, 0.999))

In [24]:
test_x = x_test_tensor[:1000]
test_y = y_test_tensor[:1000]

In [25]:
# sortmax 结果转 onehot
def props_to_onehot(props):
    if isinstance(props, list):
        props = np.array(props)
    a = np.argmax(props, axis=1)
    b = np.zeros((len(a), props.shape[1]))
    b[np.arange(len(a)), a] = 1
    return b

# a = props_to_onehot(test_output.detach().numpy())

In [34]:
num_iter = 0

for epoch in range(train_epoch):
    for b_x, b_y in train_loader:   # gives batch data, normalize x when iterate train_loader
        C.train()
        fb_x = b_x.float()
        output = C(fb_x)       # cnn output
        output = output.to(torch.float64)
#         print("output: ")
#         print(output)
#         print(b_y)
       # loss = BCE_loss(output, b_y)
        b_y = torch.topk(b_y, 1)[1].squeeze(1)
        loss = CE_loss(output, b_y)
#         print("loss: ")
#         print(loss)
#         break
        C_optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()                 # backpropagation, compute gradients
        C_optimizer.step()                # apply gradients
        
        num_iter += 1
        if num_iter%50 == 0:
            print(num_iter)

    with torch.no_grad():
        C.eval()
#             npout = props_to_onehot(output.detach().numpy())
#             accuracy = float(np.sum((npout == b_y.data.numpy()).sum(axis=1) == 8))/ float(b_y.size(0))
#             print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.numpy(), '| test accuracy: %.2f' % accuracy)
        ftest_x = test_x.float()
        test_output = C(ftest_x)
        npout = props_to_onehot(test_output.detach().numpy())
        accuracy = float(np.sum((npout == test_y.data.numpy()).sum(axis=1) == 8))/ float(test_y.size(0))
        print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.numpy(), '| test accuracy: %.2f' % accuracy)
            
#             break

50
100
150
200
250
300
350
400
450
500
550
Epoch:  0 | train loss: 1.2740 | test accuracy: 0.61
600
650
700
750
800
850
900
950
1000
1050
1100
Epoch:  1 | train loss: 1.2757 | test accuracy: 0.52
1150
1200
1250
1300
1350
1400


KeyboardInterrupt: 

In [ ]:
sum(((props_to_onehot(C(x_test_tensor.float()).detach().numpy()) == y_test_tensor.data.numpy()).sum(axis=1)) == 8)

In [ ]:
np.sum((npout == test_y.data.numpy()).sum(axis=1) == 8)

In [ ]:
test_y.size(0)

In [ ]:
test_output = C(x_test_tensor)
#print(test_output)
npout = props_to_onehot(test_output.detach().numpy())
#pred_y = torch.max(, 1)[1].data.numpy()
#print(test_y)
accuracy = float(np.sum((npout == y_test_tensor.data.numpy()).sum(axis=1) == 8))/ float(y_test_tensor.size(0))
print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.numpy(), '| test accuracy: %.2f' % accuracy)

In [ ]:
resnet50 = models.resnet50(pretrained=True)

In [ ]:
resnet50 = resnet50.to('cuda:0')

In [ ]:
optimizer = optim.Adam(resnet50.parameters())

In [ ]:
x_test_tensor[0][0][10]

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=8):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)   #strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


def ResNet18():

    return ResNet(ResidualBlock)

In [ ]:
device = torch.device("cpu")

In [ ]:
EPOCH = 10  #遍历数据集次数
pre_epoch = 0  # 定义已经遍历数据集的次数
BATCH_SIZE = 64      #批处理尺寸(batch_size)
LR = 0.01        #学习率

In [ ]:
net = ResNet18().to("cpu")
criterion = nn.CrossEntropyLoss()  #损失函数为交叉熵，多用于多分类问题nn.BCELoss()#
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4) 
#优化方式为mini-batch momentum-SGD，并采用L2正则化（权重衰减）

In [ ]:
# 训练

best_acc = 85  #2 初始化best test accuracy
print("Start Training, Resnet-18!")  # 定义遍历数据集的次数

for epoch in range(pre_epoch, EPOCH):
    print('\nEpoch: %d' % (epoch + 1))
    net.train()
    sum_loss = 0.0
    correct = 0.0
    total = 0.0
    for i, data in enumerate(trainloader, 0):
        # 准备数据
        length = len(trainloader)
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # forward + backward
        inputs = inputs.float()
        outputs = net(inputs)
        labels = labels.float()
        labels = torch.topk(labels, 1)[1].squeeze(1)
#         print(outputs[0])
#         print(labels)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 每训练1个batch打印一次loss和准确率
        sum_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += predicted.eq(labels.data).cpu().sum()
        print('[epoch:%d, iter:%d] Loss: %.03f | Acc: %.3f%% '
              % (epoch + 1, (i + 1 + epoch * length), sum_loss / (i + 1), 100. * correct / total))

# 每训练完一个epoch测试一下准确率
        if i%50 ==0:
            print("Waiting Test!")
            with torch.no_grad():
                correct = 0
                total = 0
                for data in testloader:
                    net.eval()
                    images, labels = data
                    images, labels = images.to(device), labels.to(device)
                    labels = labels.float()
                    labels = torch.topk(labels, 1)[1].squeeze(1)
                    images = images.float()
                    outputs = net(images)
                    # 取得分最高的那个类 (outputs.data的索引号)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum()
                    net.train()
                print('测试分类准确率为：%.3f%%' % (100 * correct / total))
        
print("Training Finished, TotalEPOCH=%d" % EPOCH)

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for data in testloader:
        net.eval()
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        labels = labels.float()
        labels = torch.topk(labels, 1)[1].squeeze(1)
        images = images.float()
        outputs = net(images)
        # 取得分最高的那个类 (outputs.data的索引号)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
    print('测试分类准确率为：%.3f%%' % (100 * correct / total))
    acc = 100. * correct / total

In [ ]:
correct = 0
total = 0
for data in testloader:
    net.eval()
    images, labels = data
    images, labels = images.to(device), labels.to(device)
    labels = labels.float()
    labels = torch.topk(labels, 1)[1].squeeze(1)
    images = images.float()
    outputs = net(images)
    # 取得分最高的那个类 (outputs.data的索引号)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
print('测试分类准确率为：%.3f%%' % (100 * correct / total))
acc = 100. * correct / total

In [ ]:
for data in testloader:
    net.eval()
    images, labels = data
    images, labels = images.to(device), labels.to(device)
    labels = labels.float()
    labels = torch.topk(labels, 1)[1].squeeze(1)
    images = images.float()
    outputs = net(images)
    # 取得分最高的那个类 (outputs.data的索引号)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
print('测试分类准确率为：%.3f%%' % (100 * correct / total))
acc = 100. * correct / total